In [1]:
# Use with NHL_Model_Data_Transform_v5.py
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras


rng = 69

In [2]:
df = pd.read_csv(r"C:\Users\zchodan\OneDrive - Franklin Templeton\Documents\Python\NHL_data\NHL_Data_Over_Under_Transformed.csv")
#df = pd.read_csv(r"C:\Users\zanec\OneDrive\Documents\Python\NHL_data\NHL_Data_All_Games_Transformed.csv")

In [3]:
df = df.drop(columns=['team_Home','team_Away','gameId','home_or_away_Home','home_or_away_Away','win_or_lose_Away','win_or_lose_Home','goalsTotal_Away'])

In [4]:
# df.hist(bins=50, figsize=(20,15))
# plt.show()

In [4]:
# Heatmap to visualize correlations
corr_matrix = df.corr(numeric_only=True)
# sns.heatmap(corr_matrix, cmap="PiYG")

In [5]:
corr_matrix['goalsTotal_Home'].sort_values(ascending=False)

goalsTotal_Home              1.000000
goalsAgainstAvg_Home         0.052154
goalsForAvg_Away             0.048619
goalsAgainstAvg_Away         0.040320
reboundsForAvg_Away          0.020040
reboundsForAvg_Home          0.016664
penaltiesForTotal            0.011857
fenwickPercentageAvg_Away    0.003288
penaltiesAgainstTotal        0.001298
pointsFromGameAvg_Away      -0.002941
goalsForAvg_Home            -0.003039
seasonPointsPerGame_Away    -0.006146
hitsDiffAvg_Away            -0.007814
fenwickPercentageAvg_Home   -0.008797
seasonPointsPerGame_Home    -0.022595
pointsFromGameAvg_Home      -0.024717
hitsDiffAvg_Home            -0.029000
Name: goalsTotal_Home, dtype: float64

In [7]:
# Visualize distributions of attributes
# from pandas.plotting import scatter_matrix

# attributes = ['fenwickPercentageAvg_Home','goalDiffAvg_Home','seasonPointsPerGame_Home','seasonPointsPerGame_Away']
# scatter_matrix(df[attributes], figsize=(12,8));

In [6]:
# These are tentative just to test
df_Final = df
# df_Final = df.drop(columns=['corsiPercentageAvg_Away','penaltiesAgainstTotal','shotsOnGoalDiffAvg_Away'])

In [7]:
from sklearn.model_selection import train_test_split

# Split into 80% training data and 20% testing data
train_set, test_set = train_test_split(df_Final, test_size=0.2, random_state=rng)

nhl = train_set.drop('goalsTotal_Home', axis=1)
nhl_labels = train_set['goalsTotal_Home'].copy()

X_train = train_set.drop('goalsTotal_Home', axis=1)
Y_train = train_set['goalsTotal_Home'].copy()

X_valid = train_set.drop('goalsTotal_Home', axis=1)
Y_valid = train_set['goalsTotal_Home'].copy()

X_test = test_set.drop('goalsTotal_Home', axis=1)
Y_test = test_set['goalsTotal_Home'].copy()

df_Final = df_Final.drop('goalsTotal_Home', axis=1)

In [15]:
# Pipeline constructor used to run transformation steps in order
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler()),
])

X_train_prepared= num_pipeline.fit_transform(X_train)
X_valid_prepared= num_pipeline.fit_transform(X_valid)
X_test_prepared= num_pipeline.fit_transform(X_test)


In [17]:
def build_model(n_neurons=50, n_hidden=3, dropout_rate=0.4, learning_rate=1e-3, input_shape=[X_train.shape[1]]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons,activation='selu'))
        model.add(keras.layers.Dropout(dropout_rate))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,loss='mean_squared_error',metrics=['mae'])
    return model

In [19]:
from scikeras.wrappers import KerasRegressor

# Seems like you need to specify some items again when using the scikeras wrapper
keras_reg = KerasRegressor(model=build_model, n_hidden=1, n_neurons=30, learning_rate=3e-3)


In [48]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    'model__n_hidden': [0,1,2,3,4,5],
    'model__dropout_rate': [0,.1,.2,.3,.4],
    'model__n_neurons': np.arange(1,50),
    'model__learning_rate' : reciprocal(1e-5, 1e-2)
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=2, cv=2, scoring='neg_mean_squared_error')

early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
rnd_search_cv.fit(X_train_prepared,Y_train, epochs=25,
                 validation_data = (X_valid_prepared, Y_valid),
                 callbacks=[early_stopping_cb])

RandomizedSearchCV(cv=2,
                   estimator=KerasRegressor(batch_size=5, epochs=25, loss='mean_squared_error', metrics=['mae'], model=<function build_model at 0x00000262CBDE60C0>, optimizer='adam', random_state=42, verbose=0),
                   n_iter=2,
                   param_distributions={'model__dropout_rate': [0, 0.1, 0.2,
                                                                0.3, 0.4],
                                        'model__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000262D991DEE0>,
                                        'model__n_hidden': [0, 1, 2, 3, 4, 5],
                                        'model__n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])},
                   scoring='neg_mean_squared_error')

68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1121 - val_loss: 0.8934
Epoch 86/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1341 - val_loss: 0.9704
Epoch 87/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0983 - val_loss: 0.9500
Epoch 88/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1421 - val_loss: 1.0421
Epoch 89/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1920 - val_loss: 0.9083
Epoch 90/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0717 - val_loss: 0.9129
Epoch 91/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0776 - val_loss: 0.9902
Epoch 92/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0917 - val_loss: 1.0052
Epoch 93/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0884 - val_loss: 1.0770
Epoch 94/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1641 - val_loss: 1.0252
Epoch 95/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2050 - val_loss: 1.0105
Epoch 96/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1265 

Epoch 53/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5871 - val_loss: 0.5961
Epoch 54/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5903 - val_loss: 0.5958
Epoch 55/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5979 - val_loss: 0.5956
Epoch 56/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5886 - val_loss: 0.5954
Epoch 57/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5985 - val_loss: 0.5952
Epoch 58/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6011 - val_loss: 0.5950
Epoch 59/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6001 - val_loss: 0.5948
Epoch 60/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5851 - val_loss: 0.5946
Epoch 61/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5880 - val_loss: 0.5945
Epoch 62/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5910 - val_loss: 0.5943
Epoch 63/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5926 - val_loss: 0.5942
Epoch 64/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - 

68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5887 - val_loss: 0.5933
Epoch 89/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5824 - val_loss: 0.5933
Epoch 90/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5901 - val_loss: 0.5933
Epoch 91/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6014 - val_loss: 0.5933
Epoch 92/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6003 - val_loss: 0.5933
Epoch 93/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5830 - val_loss: 0.5932
Epoch 94/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5976 - val_loss: 0.5932
Epoch 95/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5891 - val_loss: 0.5932
Epoch 96/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5818 - val_loss: 0.5932
Epoch 97/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5814 - val_loss: 0.5932
Epoch 98/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5876 - val_loss: 0.5931
Epoch 99/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5847 

68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5940 - val_loss: 0.5967
Epoch 23/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5891 - val_loss: 0.5963
Epoch 24/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5987 - val_loss: 0.5959
Epoch 25/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5917 - val_loss: 0.5955
Epoch 26/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5796 - val_loss: 0.5953
Epoch 27/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5929 - val_loss: 0.5951
Epoch 28/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5937 - val_loss: 0.5948
Epoch 29/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5976 - val_loss: 0.5946
Epoch 30/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5952 - val_loss: 0.5945
Epoch 31/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5826 - val_loss: 0.5944
Epoch 32/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6000 - val_loss: 0.5942
Epoch 33/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6036 

...

68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5754 - val_loss: 0.5853
Epoch 33/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5707 - val_loss: 0.5863
Epoch 34/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5753 - val_loss: 0.5848
Epoch 35/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5855 - val_loss: 0.5854
Epoch 36/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5755 - val_loss: 0.5874
Epoch 37/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5715 - val_loss: 0.5858
Epoch 38/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.5784 - val_loss: 0.5851
Epoch 39/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5614 - val_loss: 0.5845
Epoch 40/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5708 - val_loss: 0.5857
Epoch 41/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.5790 - val_loss: 0.5852
Epoch 42/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5609 - val_loss: 0.5844
Epoch 43/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5672 

Epoch 68/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5823 - val_loss: 0.5864
Epoch 69/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5727 - val_loss: 0.5862
Epoch 70/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5812 - val_loss: 0.5861
Epoch 71/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5777 - val_loss: 0.5859
Epoch 72/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5856 - val_loss: 0.5857
Epoch 73/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5847 - val_loss: 0.5854
Epoch 74/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5714 - val_loss: 0.5853
Epoch 75/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5880 - val_loss: 0.5853
Epoch 76/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5750 - val_loss: 0.5850
Epoch 77/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5780 - val_loss: 0.5849
Epoch 78/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5703 - val_loss: 0.5847
Epoch 79/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - 

68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6872 - val_loss: 0.6639
Epoch 3/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6440 - val_loss: 0.6433
Epoch 4/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6210 - val_loss: 0.6306
Epoch 5/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6164 - val_loss: 0.6223
Epoch 6/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6073 - val_loss: 0.6166
Epoch 7/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6015 - val_loss: 0.6118
Epoch 8/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6089 - val_loss: 0.6083
Epoch 9/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5977 - val_loss: 0.6056
Epoch 10/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5936 - val_loss: 0.6034
Epoch 11/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5857 - val_loss: 0.6015
Epoch 12/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6028 - val_loss: 0.6000
Epoch 13/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5871 - val_l

102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5971 - val_loss: 0.5964
Epoch 38/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5914 - val_loss: 0.5960
Epoch 39/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6004 - val_loss: 0.5956
Epoch 40/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5959 - val_loss: 0.5952
Epoch 41/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5816 - val_loss: 0.5949
Epoch 42/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5898 - val_loss: 0.5946
Epoch 43/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5850 - val_loss: 0.5943
Epoch 44/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5924 - val_loss: 0.5941
Epoch 45/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5956 - val_loss: 0.5939
Epoch 46/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5920 - val_loss: 0.5937
Epoch 47/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6004 - val_loss: 0.5935
Epoch 48/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 

RandomizedSearchCV(cv=3, error_score='raise',
                   estimator=KerasClassifier(learning_rate=0.003, model=<function build_model at 0x000001C0900D9A80>, n_hidden=1, n_neurons=30),
                   param_distributions={'model__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001C0901192D0>,
                                        'model__n_hidden': [0, 1, 2, 3],
                                        'model__n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])})

In [17]:
#print(rnd_search_cv.best_params_)
#print(rnd_search_cv.best_score_)

In [50]:
print(rnd_search_cv.best_params_)
print(rnd_search_cv.best_score_)

{'model__dropout_rate': 0.1, 'model__learning_rate': 0.000770560481758444, 'model__n_hidden': 2, 'model__n_neurons': 43}
-5.620613727916652


In [52]:
best_model = rnd_search_cv.best_estimator_.model

In [54]:
from scikeras.wrappers import KerasRegressor

# Seems like you need to specify some items again when using the scikeras wrapper
keras_reg = KerasRegressor(model=best_model, epochs=25, batch_size=5, verbose=1, random_state=42,
                           loss='mean_squared_error',optimizer='adam',metrics=['mae'])

# Stop early if model is not getting better after # of patience epochs, restore to best model
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
#checkpoint_cb = keras.callbacks.ModelCheckpoint("Regression_Sequential.keras", save_best_only = True)

keras_reg.fit(X_train_prepared, Y_train, validation_data = (X_valid_prepared, Y_valid),
             callbacks=[early_stopping_cb])


KerasRegressor(
	model=<function build_model at 0x00000262CBDE60C0>
	build_fn=None
	warm_start=False
	random_state=42
	optimizer=adam
	loss=mean_squared_error
	metrics=['mae']
	batch_size=5
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=25
)

In [56]:
from sklearn.model_selection import cross_val_score

# Assuming X and y are your feature matrix and target vector
scores = cross_val_score(keras_reg, X_train_prepared, Y_train, cv=5, scoring='neg_mean_squared_error')

# Convert scores to positive values
mse_scores = -scores
print(f'MSE Scores: {mse_scores}')
print(f'Mean MSE: {np.mean(mse_scores)}')

MSE Scores: [5.92100081 4.95293598 5.28203431 5.53841937 5.80035661]
Mean MSE: 5.498949414379491


In [61]:
X_test_pred = keras_reg.predict(X_test_prepared)

# Evaluate the model
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(Y_test, X_test_pred)
mae = mean_absolute_error(Y_test, X_test_pred)

print(f'Test MSE: {mse}') # Sensitive to outliers
print(f'Test MAE: {mae}')

Test MSE: 5.176655788586463
Test MAE: 1.843782202763991


In [25]:
df_Predict = pd.read_csv(r"C:\Users\zchodan\OneDrive - Franklin Templeton\Documents\Python\NHL_data\NHL_Data_Over_Under_Predict.csv")
#df_Predict = pd.read_csv(r"C:\Users\zanec\OneDrive\Documents\Python\NHL_data\NHL_Data_All_Games_Predict.csv")

In [26]:
X_real_predict = df_Predict.iloc[:,2:]

np.array((df_Predict.iloc[:,1].values,keras_clf.predict(X_real_predict))).T

ValueError: Found array with 0 sample(s) (shape=(0, 14)) while a minimum of 1 is required.